In [1]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import time
import gc
from glob import glob
from tqdm.notebook import tqdm
import pickle
import logging
import json

import base64
import swifter
import gensim

from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset

pd.set_option('display.max_columns', 500)

In [2]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

import keras
from keras.preprocessing.sequence import pad_sequences

import networkx as nx
import dgl
import dgl.function as fn
from dgl.nn.pytorch import GraphConv

from pytorch_metric_learning import losses

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,DistilBertTokenizer,DistilBertModel
)

print(transformers.__version__)

Using TensorFlow backend.
Using backend: pytorch
ERROR:root:The testing module requires faiss. You can install the GPU version with the command 'conda install faiss-gpu -c pytorch' 
                        or the CPU version with 'conda install faiss-cpu -c pytorch'. Learn more at https://github.com/facebookresearch/faiss/blob/master/INSTALL.md


2.8.0


In [3]:
DATA_DIR = '/home/m_fujitsuka/mnt/data/'
FEATURE_DIR = '/home/m_fujitsuka/mnt/fujitsuka/features/'
LOG_DIR = '/home/m_fujitsuka/mnt/fujitsuka/log/'
W2V_DIR = '/home/m_fujitsuka/mnt/fujitsuka/w2v/'
MODEL_DIR = '/home/m_fujitsuka/mnt/fujitsuka/model/'
SUBMIT_DIR = '/home/m_fujitsuka/mnt/fujitsuka/prediction_result/'

In [4]:
train = pd.read_pickle(FEATURE_DIR+'train.pkl')
valid = pd.read_pickle(FEATURE_DIR+'valid.pkl')
testA = pd.read_pickle(FEATURE_DIR+'testA.pkl')

In [5]:
print(train.shape)
train.head()

(3000000, 14)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_area_normalized,class_labels_vec,center_normalized,adj_mat,box_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...",others others,treble popular reed,923271,"[0.9760765550239234, 0.2421788737578212]","[-0.16868001, -0.257395, -0.53158, -0.04389, -...","[[0.5023923444976076, 0.5], [0.540669856459330...","[[0.0, 2.7414992979958495], [2.741499297995849...","[[0.0, 0.014354066985645933, 1.0, 0.9904306220..."
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",others,compatible ink cartridges,234031,[0.7315573770491803],"[-0.16868001, -0.257395, -0.53158, -0.04389, -...","[[0.5, 0.5]]",[[0.0]],"[[0.0625, 0.08196721311475409, 0.9375, 0.91803..."
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...","makeup, perfume, beauty tools and essential oils",check new look facial cream,199926,[0.2241333333333333],"[-0.122048564, 0.036015432, -0.06529144, 0.100...","[[0.48733333333333334, 0.4673333333333333]]",[[0.0]],"[[0.05733333333333333, 0.3506666666666667, 0.8..."
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....",others others others others others,royal commemorative stamp,772725,"[0.11563802083333333, 0.120419921875, 0.133255...","[-0.16868001, -0.257395, -0.53158, -0.04389, -...","[[0.825, 0.6937500000000001], [0.17109375, 0.7...","[[0.0, 82.60384316863635, 50.596186563989285, ...","[[0.5208333333333334, 0.6578125, 0.86666666666..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","others snacks, nuts, liquor and tea snacks, nu...",calendula baby shower gel,362576,"[0.25558437500000003, 0.0356, 0.03279375, 0.21...","[-0.03929567, -0.14148633, -0.143134, -0.30245...","[[0.49812500000000004, 0.53375], [0.9137500000...","[[0.0, 3.7297644696196075, 3.456343151840976, ...","[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8..."


### Negative Sample作成

In [6]:
# 今回は、まず4割で！！
train = train.sample(frac=0.4, random_state=42)
train.shape

(1200000, 14)

In [7]:
# train['label'] = 1

# random.seed(42)
# ind = list(train.index)
# random.shuffle(ind)

# train_neg = train.copy(deep=True)
# train_neg['label'] = 0
# train_neg['query'] = list(train_neg['query'].loc[ind])[:]
# train_neg.head()

# train = pd.concat([train, train_neg], axis=0)
# train = train.reset_index(drop=True)
# print(train.shape)

### 初期設定

In [8]:
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
sc = logging.StreamHandler()
logger.addHandler(sc)
fh = logging.FileHandler(LOG_DIR+'20200506_each.log')
logger.addHandler(fh)

In [9]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


### DataLoader

In [11]:
MAX_LEN = 20

class KDDDataset(Dataset):

    def __init__(self, df, train_mode=True, transform=None):
        self.df = df
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        #self.w2v_tokenizer = tokenizer_w2v
        self.train_mode = train_mode
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        ######################
        # query side
        ######################
        query = row['query']
        inputs_query = self.bert_tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )
        
        ids_query = inputs_query["input_ids"]
        token_type_ids_query = inputs_query["token_type_ids"]
        mask_query = inputs_query["attention_mask"]
        
        padding_len = MAX_LEN - len(ids_query)
        ids_query = ids_query + ([0]*padding_len)
        token_type_ids_query = token_type_ids_query + ([0]*padding_len)
        mask_query = mask_query + ([0]*padding_len)
        
        
        ######################
        # image side
        ######################
        # box_areaの大きい順にソート
        box_order = np.argsort(row.box_area_normalized)[::-1]

        faetures_ordered = row.features[box_order, :]
        box_ordered = row.box_normalized[box_order, :]
        center_ordered = row.center_normalized[box_order, :]
        area_ordered = row.box_area_normalized[box_order]

        n_box = row.num_boxes
        adj_mat = np.zeros((n_box, n_box))
        if n_box!=1:
            X = center_ordered[:,0]
            Y = center_ordered[:,1]
            for i in range(1, n_box):
                d = np.sqrt((X[0] - X[i])**2 + (Y[0] - Y[i])**2)
                adj_mat[0, i] = 1/d
                adj_mat[i, 0] = 1/d

        # adj_mat = row['adj_mat']
        # features = row['features']
        # box = row['box_normalized']
        # box_area = row['box_area_normalized']
        
        nxg = nx.from_numpy_matrix(adj_mat)
        g = dgl.DGLGraph()
        g.from_networkx(nxg)
        g.edata['weight'] = torch.tensor(nx.adjacency_matrix(nxg).data, dtype=torch.float).to(device)
        g.ndata['h'] = torch.cat([torch.tensor(faetures_ordered, dtype=torch.float),torch.tensor(box_ordered, dtype=torch.float),torch.tensor(area_ordered.reshape(-1,1), dtype=torch.float)], dim=-1).to(device)
        g.ndata['w'] = torch.tensor(area_ordered, dtype=torch.float).to(device)
        
        # class_labels
        #class_labels_vec = row['class_labels_vec']  # 含めると悪化
        
        
        if self.train_mode:
            # positive, negative label
            #labels = row['label']
            
            
            return g, {
                'ids_query': torch.tensor(ids_query, dtype=torch.long),
                'mask_query': torch.tensor(mask_query, dtype=torch.long),
                'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
                #'class_labels_vec': torch.tensor(class_labels_vec, dtype=torch.float),
                #'label': torch.tensor(labels, dtype=torch.float),
            }
        
        else:
            return g, {
                'ids_query': torch.tensor(ids_query, dtype=torch.long),
                'mask_query': torch.tensor(mask_query, dtype=torch.long),
                'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
                #'class_labels_vec': torch.tensor(class_labels_vec, dtype=torch.float),
            }

In [12]:
def collate(samples):
    # input samples は、ペアのリスト
    #  [(graph, {features}),...], バッチサイズの長さのリスト
    # グラフは、隣接行列を対角につなげて1つのバッチにする
    # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
    graphs, features_dict = map(list, zip(*samples))
    
    batched_graph = dgl.batch(graphs)
    
    batched_features = {}
    labels = []
    for i, feat in enumerate(features_dict):
        if i==0:
            for (key, val) in feat.items():
                batched_features[key] = val.view(1,-1)
               
        else:
            for (key, val) in feat.items():
                batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
                    
    return batched_graph, batched_features


def collate_test(samples):
    # input samples は、ペアのリスト
    #  [(graph, {features}),...], バッチサイズの長さのリスト
    # グラフは、隣接行列を対角につなげて1つのバッチにする
    # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
    graphs, features_dict = map(list, zip(*samples))
    
    batched_graph = dgl.batch(graphs)
    
    batched_features = {}
    labels = []
    for i, feat in enumerate(features_dict):
        if i==0:
            for (key, val) in feat.items():
                batched_features[key] = val.view(1,-1)
        else:
            for (key, val) in feat.items():
                batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
        
    return batched_graph, batched_features

In [13]:
# # dataloaderの挙動確認

# dataset_train = KDDDataset(train, train_mode=True)
# train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)

# for i, (graph, features) in enumerate(train_loader):
#     if i<1:
#         print(graph.batch_size)
#         print(graph)
#         print(features['ids_query'].shape)
#         print(features['mask_query'].shape)
#         print(features['token_type_ids_query'].shape)
#         print(features['class_labels_vec'].shape)
#         print(features['label'].shape)

#     else:
#         break

### Model

In [14]:
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.fc = nn.Linear(in_feats, out_feats)
        self.activation = activation
    
    def message_func(self, edges):
        """edgeを通して，どのようなメッセージをどう渡すかを決める関数"""
        weight = edges.data['weight'].view(-1, 1)
        messages = edges.src['h'] * weight
        return {'m': messages}
    
    def reduce_func(self, nodes):
        """渡されたメッセージをどのように集約するかを決める関数"""
        messages = nodes.mailbox['m']
        h = torch.sum(messages, dim=1)
        return {'h': h}
    
    def node_apply_func(self, nodes):
        """ノードごとに作用させる関数"""
        h = self.fc(nodes.data['h'])
        h = self.activation(h)
        return {'h': h}

    def forward(self, g, h):
        g.ndata['h'] = h
        g.apply_nodes(self.node_apply_func)
        g.update_all(self.message_func, self.reduce_func)
        return g.ndata.pop('h')  # ndata['h']が削除されるが，返り値としては，ndata['h']となる

In [15]:
class QueryModel(nn.Module):
    def __init__(self):
        super(QueryModel, self).__init__()
        self.model_name = 'QueryModel'

        # query
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.fc_bert = nn.Linear(768, 128)
        #self.fc_bert2 = nn.Linear(128, 64)


    def forward(self, ids, token_type_ids, mask):
        
        ##############################
        # query
        ##############################
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=token_type_ids, attention_mask=mask)
        out_query = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()
        out_query = F.dropout(out_query, p=0.3, training=self.training)
        out_query = F.relu(self.fc_bert(out_query))
        #out_query = F.relu(self.fc_bert2(out_query))
        
        return out_query

In [16]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.model_name = 'ImageModel'

        # image
        self.gcn1 = GCN(2048+4+1, 512, F.relu)
        self.gcn2 = GCN(512, 256, F.relu)
        self.fc1 = nn.Linear(256, 128)
        #self.fc2 = nn.Linear(128, 64)


    def forward(self, g):
        
        ##############################
        # image
        ##############################
        h = g.ndata.pop('h')
        h = self.gcn1(g, h)
        h = self.gcn2(g, h)
        g.ndata['h'] = h
        
        for i, graph in enumerate(dgl.unbatch(g)):
            if i==0:
                out_image = graph.ndata['h'][0].unsqueeze(0)
            else:
                out_image = torch.cat([out_image, graph.ndata['h'][0].unsqueeze(0)], dim=0)


        # Calculate graph representation by averaging all the node representations.
        #out_image = dgl.mean_nodes(g, 'h', 'w')  # wでの重み付き平均
        # out_image = dgl.max_nodes(g, 'h')
        
        #out_image = torch.cat([out_image, class_labels_vec], dim=-1)
        
        out_image = F.dropout(out_image, p=0.2, training=self.training)
        out_image = F.relu(self.fc1(out_image))
        #out_image = F.relu(self.fc2(out_image))
        
        return out_image

In [17]:
class KDDModel(nn.Module):
    def __init__(self):
        super(KDDModel, self).__init__()
        self.model_name = 'KDDModel'

        self.model_query = QueryModel()
        self.model_image = ImageModel()


    def forward(self, ids, token_type_ids, mask, g):
        
        out_query = self.model_query(ids, token_type_ids, mask)
        out_image = self.model_image(g)
        
        return out_query, out_image

### Training

In [18]:
def train_model(train_loader, model, optimizer, criterion): #, scheduler):
    model.train()
    avg_loss = 0.
    avg_score = 0.
    for idx, (graph, features) in enumerate(tqdm(train_loader)):
        ids_query = features['ids_query'].to(device)
        mask_query = features['mask_query'].to(device)
        token_type_ids_query = features['token_type_ids_query'].to(device)
        #labels = features['label'].to(device)

        out_query, out_image = model(ids_query, token_type_ids_query, mask_query, graph)
        
        loss = criterion(out_query, out_image)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item() / len(train_loader)

        del ids_query,mask_query,token_type_ids_query,out_query,out_image,loss

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss



def test_model(test_loader, model):
    model.eval()
    
    out_list = []   
    with torch.no_grad():
        for idx, (graph, features) in enumerate(tqdm(test_loader)):
            ids_query = features['ids_query'].to(device)
            mask_query = features['mask_query'].to(device)
            token_type_ids_query = features['token_type_ids_query'].to(device)

            out_query, out_image = model(ids_query, token_type_ids_query, mask_query, graph)

            out = F.cosine_similarity(out_query, out_image)

            out_list.append(out)
            del ids_query,mask_query,token_type_ids_query,out_query,out_image
        
    torch.cuda.empty_cache()
    gc.collect()
    return out_list

In [19]:
# def train_model(train_loader, model, optimizer, criterion): #, scheduler):
#     model.train()
#     avg_loss = 0.
#     avg_score = 0.
#     for idx, (graph, features) in enumerate(tqdm(train_loader)):
#         ids_query = features['ids_query'].to(device)
#         mask_query = features['mask_query'].to(device)
#         token_type_ids_query = features['token_type_ids_query'].to(device)
#         labels = features['label'].to(device)

#         out = model(ids_query, token_type_ids_query, mask_query, graph)
        
#         loss = criterion(out, labels)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         avg_loss += loss.item() / len(train_loader)

#         del ids_query,mask_query,token_type_ids_query,labels,out,loss

#     torch.cuda.empty_cache()
#     gc.collect()
#     return avg_loss



# def test_model(test_loader, model):
#     model.eval()
    
#     out_list = []   
#     with torch.no_grad():
#         for idx, (graph, features) in enumerate(tqdm(test_loader)):
#             ids_query = features['ids_query'].to(device)
#             mask_query = features['mask_query'].to(device)
#             token_type_ids_query = features['token_type_ids_query'].to(device)

#             out_query = model.model_query(ids_query, token_type_ids_query, mask_query)
#             out_image = model.model_image(graph)
#             out = F.cosine_similarity(out_query, out_image)

#             out_list.append(out)
#             del ids_query,mask_query,token_type_ids_query,out
        
#     torch.cuda.empty_cache()
#     gc.collect()
#     return out_list

In [20]:
# compute dcg@k for a single sample
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(3, r.size + 2)))
    return 0.


# compute ndcg@k (dcg@k / idcg@k) for a single sample
def get_ndcg(r, ref, k):
    dcg_max = dcg_at_k(ref, k)
    if not dcg_max:
        return 0.
    dcg = dcg_at_k(r, k)
    return dcg / dcg_max


def make_score(df_test, out_list):
    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    predictions = {}
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        predictions[f'{q_id}'] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values.astype(str)
    
    
    # read ground-truth
    reference = json.load(open(DATA_DIR+'valid_answer.json'))

    # compute score for each query
    k = 5
    score_sum = 0.
    for qid in reference.keys():
        ground_truth_ids = set([str(pid) for pid in reference[qid]])
        ref_vec = [1.0] * len(ground_truth_ids)
        pred_vec = [1.0 if pid in ground_truth_ids else 0.0 for pid in predictions[qid]]
        score_sum += get_ndcg(pred_vec, ref_vec, k)
    # the higher score, the better
    score = score_sum / len(reference)
    
    return score

In [ ]:
model = KDDModel().to(device)

# if device == 'cuda':
#     model = torch.nn.DataParallel(model) # make parallel
#     cudnn.benchmark = True


optimizer = torch.optim.Adam(model.parameters())

# optimizer = torch.optim.Adam([
#                         {'params': model.model_query.parameters()},
#                         {'params': model.model_image.parameters()}
#                     ])


#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.CosineSimilarity()
criterion = nn.MSELoss()
#criterion = nn.CosineEmbeddingLoss()
#criterion = losses.TripletMarginLoss(margin=0.1)

loss_list_epoch_train = []
score_list_epoch_valid = []
score_best = 0
patience = 0
EPOCHS = 10
for epoch in range(EPOCHS):

    torch.cuda.empty_cache()
    start_time   = time.time()

    dataset_train = KDDDataset(train, train_mode=True)
    dataset_valid = KDDDataset(valid, train_mode=False)
    train_loader = DataLoader(dataset_train, batch_size=256, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)
    valid_loader = DataLoader(dataset_valid, batch_size=256, shuffle=False, num_workers=0, drop_last=False, collate_fn=collate_test)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=EPOCHS)

    loss_train = train_model(train_loader, model, optimizer, criterion) #, scheduler)
    #scheduler.step()
    out_valid = test_model(valid_loader, model)
    score_valid = make_score(valid, out_valid)

    loss_list_epoch_train.append(loss_train)
    score_list_epoch_valid.append(score_valid)

    logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_score: {score_valid}, time: {(time.time()-start_time)}')

    # Eearly Stopping
    if score_valid > score_best:
        score_best = score_valid
        best_param = model.state_dict()
        torch.save(best_param, MODEL_DIR+f'best_param.pt')
        patience = 0
    else:
        patience += 1
        if patience > 2:
            del train_loader, valid_loader, loss_train, out_valid
            torch.cuda.empty_cache()
            gc.collect()
            break

    del train_loader, valid_loader, loss_train, out_valid
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# model = KDDModel().to(device)

# # if device == 'cuda':
# #     model = torch.nn.DataParallel(model) # make parallel
# #     cudnn.benchmark = True


# # optimizer = torch.optim.Adam(model.parameters())

# optimizer = torch.optim.Adam([
#                         {'params': model.model_query.parameters()},
#                         {'params': model.model_image.parameters()}
#                     ])


# #criterion = nn.BCEWithLogitsLoss()
# #criterion = nn.CosineSimilarity()
# #criterion = nn.CosineEmbeddingLoss()
# criterion = losses.TripletMarginLoss(margin=0.1)

# loss_list_epoch_train = []
# score_list_epoch_valid = []
# score_best = 0
# patience = 0
# EPOCHS = 10
# for epoch in range(EPOCHS):

#     torch.cuda.empty_cache()
#     start_time   = time.time()

#     dataset_train = KDDDataset(train, train_mode=True)
#     dataset_valid = KDDDataset(valid, train_mode=False)
#     train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)
#     valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=False, collate_fn=collate_test)
#     #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)
#     #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=EPOCHS)

#     loss_train = train_model(train_loader, model, optimizer, criterion) #, scheduler)
#     #scheduler.step()
#     out_valid = test_model(valid_loader, model)
#     score_valid = make_score(valid, out_valid)

#     loss_list_epoch_train.append(loss_train)
#     score_list_epoch_valid.append(score_valid)

#     logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_score: {score_valid}, time: {(time.time()-start_time)}')

#     # Eearly Stopping
#     if score_valid > score_best:
#         score_best = score_valid
#         best_param = model.state_dict()
#         torch.save(best_param, MODEL_DIR+f'best_param.pt')
#         patience = 0
#     else:
#         patience += 1
#         if patience > 2:
#             del train_loader, valid_loader, loss_train, out_valid
#             torch.cuda.empty_cache()
#             gc.collect()
#             break

#     del train_loader, valid_loader, loss_train, out_valid
#     torch.cuda.empty_cache()
#     gc.collect()

### Inference

In [ ]:
model = KDDModel().to(device)
model.load_state_dict(torch.load(MODEL_DIR+f'best_param.pt'))

In [ ]:
def make_submit_df(df_test, model):
    dataset_test = KDDDataset(df_test, train_mode=False)
    test_loader = DataLoader(dataset_test, batch_size=256, shuffle=False, num_workers=0, drop_last=False, collate_fn=collate_test)
    out_list = test_model(test_loader, model)

    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    submit_df = pd.DataFrame(columns=['query-id','product1','product2','product3','product4','product5'])
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        submit_df.loc[i, 'query-id'] = q_id
        submit_df.iloc[i, 1:] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values
    submit_df = submit_df.astype(int)
    submit_df = submit_df.sort_values(by='query-id')
    submit_df = submit_df.reset_index(drop=True)
    
    return submit_df

In [ ]:
submit_df = make_submit_df(testA, model)
submit_df.to_csv(SUBMIT_DIR+'submission.csv', index=False)